In [ ]:
from time import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp
from Dataset import Dataset
from evaluate import evaluate_model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
print(f"Pytorch 版本为:{torch.__version__}")

# GMF

In [ ]:
num_factors = 8 #嵌入向量维度
num_negatives = 4 #负实例数量
learner = 'adam' #学习器
learning_rate = 0.001 #学习率0.001
epochs = 20 #100轮次
batch_size = 256 #批次256
verbose = 1 #每1次迭代显示一次性能，默认为 1

topK = 10
evaluation_threads = 1 

# 利用源代码的Dataset.py加载数据
t1 = time()
dataset = Dataset('Data/ml-1m')
train, testRatings, testNegatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives
num_users, num_items = train.shape
print("Load data done [%.1f s]. #user=%d, #item=%d, #train=%d, #test=%d" 
      %(time()-t1, num_users, num_items, train.nnz, len(testRatings)))

In [ ]:
# 建立GMF模型
class GMF(nn.Module):
    def __init__(self, num_users, num_items, latent_dim):
        super(GMF, self).__init__()
        self.user_embedding = nn.Embedding(num_users, latent_dim)
        self.item_embedding = nn.Embedding(num_items, latent_dim)
        
        # Initialize embeddings
        nn.init.normal_(self.user_embedding.weight, mean=0, std=0.01)
        nn.init.normal_(self.item_embedding.weight, mean=0, std=0.01)
        
        self.output = nn.Linear(latent_dim, 1)
        nn.init.kaiming_uniform_(self.output.weight, nonlinearity='sigmoid')

    def forward(self, user_input, item_input):
        user_latent = self.user_embedding(user_input).squeeze(1)
        item_latent = self.item_embedding(item_input).squeeze(1)
        predict_vector = user_latent * item_latent
        
        # Final prediction layer
        prediction = torch.sigmoid(self.output(predict_vector))
        return prediction.squeeze()


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GMF(num_users, num_items, num_factors).to(device)
criterion = nn.BCELoss()
if learner.lower() == "adagrad": 
    optimizer = optim.Adagrad(model.parameters(), lr=learning_rate)
elif learner.lower() == "rmsprop":
    optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
elif learner.lower() == "adam":
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
else:
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
print(model)

# Init performance
t1 = time()
(hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK, evaluation_threads, device)
hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
print('Init: HR = %.4f, NDCG = %.4f\t [%.1f s]' % (hr, ndcg, time()-t1))

In [ ]:
def get_train_instances(train, num_negatives):
    user_input, item_input, labels = [], [], []
    num_users = train.shape[0]
    num_items = train.shape[1]  # 假设 train 是用户-物品矩阵

    for (u, i) in train.keys():
        # 正例
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        # 负例
        for _ in range(num_negatives):
            j = np.random.randint(num_items)
            while (u, j) in train:
                j = np.random.randint(num_items)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    return user_input, item_input, labels

# Train model
best_hr, best_ndcg, best_iter = -1, -1, -1
for epoch in range(epochs):
    t1 = time()
    # Generate training instances
    user_input, item_input, labels = get_train_instances(train, num_negatives)

    # Create DataLoader
    train_dataset = TensorDataset(torch.LongTensor(user_input), torch.LongTensor(item_input), torch.FloatTensor(labels))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Training
    model.train()
    total_loss = 0
    for batch_user, batch_item, batch_label in train_loader:
        batch_user, batch_item, batch_label = batch_user.to(device), batch_item.to(device), batch_label.to(device)
        optimizer.zero_grad()
        output = model(batch_user, batch_item).squeeze() 
        loss = criterion(output, batch_label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    t2 = time()
    
    # Evaluation
    if epoch % verbose == 0:
        model.eval()
        hits, ndcgs = evaluate_model(model, testRatings, testNegatives, topK, evaluation_threads,device)
        hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
        print(f'Iteration {epoch} [{t2-t1:.1f} s]: HR = {hr:.4f}, NDCG = {ndcg:.4f}, loss = {total_loss/len(train_loader):.4f} [{time()-t2:.1f} s]')
        if hr > best_hr:
            best_hr, best_ndcg, best_iter = hr, ndcg, epoch

print(f"End. Best Iteration {best_iter}:  HR = {best_hr:.4f}, NDCG = {best_ndcg:.4f}.")


# MLP

In [ ]:
num_negatives = 4 #负实例数量
learner = 'adam' #学习器
learning_rate = 0.001 #学习率0.001
epochs = 50 #100轮次
batch_size = 256 #批次256
verbose = 1 #每1次迭代显示一次性能，默认为 1
layers = [64,32,16,8]
reg_layers = [0,0,0,0]

topK = 10
evaluation_threads = 1 

# 利用源代码的Dataset.py加载数据
t1 = time()
dataset = Dataset('Data/ml-1m')
train, testRatings, testNegatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives
num_users, num_items = train.shape
print("Load data done [%.1f s]. #user=%d, #item=%d, #train=%d, #test=%d" 
      %(time()-t1, num_users, num_items, train.nnz, len(testRatings)))

In [ ]:
class MLP(nn.Module):
    def __init__(self, num_users, num_items, layers=[20, 10], reg_layers=[0, 0]):
        super(MLP, self).__init__()
        assert len(layers) == len(reg_layers)
        self.num_layer = len(layers)
        
        # Embedding layers
        self.user_embedding = nn.Embedding(num_users, int(layers[0] / 2))
        self.item_embedding = nn.Embedding(num_items, int(layers[0] / 2))
        
        nn.init.normal_(self.user_embedding.weight, mean=0, std=0.01)
        nn.init.normal_(self.item_embedding.weight, mean=0, std=0.01)
        
        # MLP layers
        mlp_layers = []
        for i in range(1, self.num_layer):
            input_size = layers[i-1]
            output_size = layers[i]
            mlp_layers.append(nn.Linear(input_size, output_size))
            mlp_layers.append(nn.ReLU())
            if reg_layers[i] > 0:
                mlp_layers.append(nn.Dropout(reg_layers[i]))
        self.mlp_layers = nn.Sequential(*mlp_layers)
        
        # Output layer
        self.output = nn.Linear(layers[-1], 1)
        nn.init.kaiming_uniform_(self.output.weight, nonlinearity='sigmoid')

    def forward(self, user_input, item_input):
        user_latent = self.user_embedding(user_input).squeeze(1)
        item_latent = self.item_embedding(item_input).squeeze(1)
        
        # Concatenate user and item embeddings
        vector = torch.cat([user_latent, item_latent], dim=-1)
        
        # Pass through MLP layers
        vector = self.mlp_layers(vector)
        
        # Final prediction layer
        prediction = torch.sigmoid(self.output(vector))
        return prediction.squeeze()

    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MLP(num_users, num_items, layers, reg_layers).to(device)
criterion = nn.BCELoss()
if learner.lower() == "adagrad": 
    optimizer = optim.Adagrad(model.parameters(), lr=learning_rate)
elif learner.lower() == "rmsprop":
    optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
elif learner.lower() == "adam":
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
else:
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
print(model)

# Init performance
t1 = time()
(hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK, evaluation_threads, device)
hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
print('Init: HR = %.4f, NDCG = %.4f\t [%.1f s]' % (hr, ndcg, time()-t1))

In [ ]:
# Train model
best_hr, best_ndcg, best_iter = -1, -1, -1
for epoch in range(epochs):
    t1 = time()
    # Generate training instances
    user_input, item_input, labels = get_train_instances(train, num_negatives)

    # Create DataLoader
    train_dataset = TensorDataset(torch.LongTensor(user_input), torch.LongTensor(item_input), torch.FloatTensor(labels))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Training
    model.train()
    total_loss = 0
    for batch_user, batch_item, batch_label in train_loader:
        batch_user, batch_item, batch_label = batch_user.to(device), batch_item.to(device), batch_label.to(device)
        optimizer.zero_grad()
        output = model(batch_user, batch_item).squeeze() 
        loss = criterion(output, batch_label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    t2 = time()
    
    # Evaluation
    if epoch % verbose == 0:
        model.eval()
        hits, ndcgs = evaluate_model(model, testRatings, testNegatives, topK, evaluation_threads,device)
        hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
        print(f'Iteration {epoch} [{t2-t1:.1f} s]: HR = {hr:.4f}, NDCG = {ndcg:.4f}, loss = {total_loss/len(train_loader):.4f} [{time()-t2:.1f} s]')
        if hr > best_hr:
            best_hr, best_ndcg, best_iter = hr, ndcg, epoch

print(f"End. Best Iteration {best_iter}:  HR = {best_hr:.4f}, NDCG = {best_ndcg:.4f}.")


# NeuMF


In [ ]:
num_epochs = 20
batch_size = 256
mf_dim = 128
layers = [256,128,64,32]
reg_mf = 0
reg_layers = [0,0,0,0]
num_negatives = 4
learning_rate = 0.001
learner = 'adam'
verbose = 1
mf_pretrain = ''
mlp_pretrain = ''

topK = 10
evaluation_threads = 1
# 利用源代码的Dataset.py加载数据
t1 = time()
dataset = Dataset('Data/ml-1m')
train, testRatings, testNegatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives
num_users, num_items = train.shape
print("Load data done [%.1f s]. #user=%d, #item=%d, #train=%d, #test=%d" 
      %(time()-t1, num_users, num_items, train.nnz, len(testRatings)))

In [ ]:
class NeuMF(nn.Module):
    def __init__(self, num_users, num_items, mf_dim=10, layers=[10], reg_layers=[0], reg_mf=0):
        super(NeuMF, self).__init__()
        assert len(layers) == len(reg_layers)
        self.mf_dim = mf_dim
        self.num_layer = len(layers)
        
        # MF embeddings
        self.mf_user_embedding = nn.Embedding(num_users, mf_dim)
        self.mf_item_embedding = nn.Embedding(num_items, mf_dim)
        
        # Initialize MF embeddings
        nn.init.normal_(self.mf_user_embedding.weight, mean=0, std=0.01)
        nn.init.normal_(self.mf_item_embedding.weight, mean=0, std=0.01)
        
        # MLP embeddings
        self.mlp_user_embedding = nn.Embedding(num_users, layers[0] // 2)
        self.mlp_item_embedding = nn.Embedding(num_items, layers[0] // 2)
        
        # Initialize MLP embeddings
        nn.init.normal_(self.mlp_user_embedding.weight, mean=0, std=0.01)
        nn.init.normal_(self.mlp_item_embedding.weight, mean=0, std=0.01)
        
        # MLP layers
        mlp_layers = []
        for i in range(1, self.num_layer):
            mlp_layers.append(nn.Linear(layers[i-1], layers[i]))
            mlp_layers.append(nn.ReLU())
            if reg_layers[i] > 0:
                mlp_layers.append(nn.Dropout(reg_layers[i]))
        self.mlp_layers = nn.Sequential(*mlp_layers)
        
        # Final prediction layer
        predict_size = mf_dim + layers[-1]
        self.output = nn.Linear(predict_size, 1)
        nn.init.kaiming_uniform_(self.output.weight, nonlinearity='sigmoid')

    def forward(self, user_input, item_input):
        # MF part
        mf_user_latent = self.mf_user_embedding(user_input)
        mf_item_latent = self.mf_item_embedding(item_input)
        mf_vector = mf_user_latent * mf_item_latent
        
        # MLP part
        mlp_user_latent = self.mlp_user_embedding(user_input)
        mlp_item_latent = self.mlp_item_embedding(item_input)
        mlp_vector = torch.cat((mlp_user_latent, mlp_item_latent), dim=-1)
        mlp_vector = self.mlp_layers(mlp_vector)
        
        # Concatenate MF and MLP parts
        predict_vector = torch.cat((mf_vector, mlp_vector), dim=-1)
        
        # Final prediction layer
        prediction = torch.sigmoid(self.output(predict_vector))
        return prediction.squeeze()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuMF(num_users, num_items, mf_dim, layers, reg_layers, reg_mf).to(device)
criterion = nn.BCELoss()

if learner.lower() == "adagrad": 
    optimizer = optim.Adagrad(model.parameters(), lr=learning_rate)
elif learner.lower() == "rmsprop":
    optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
elif learner.lower() == "adam":
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
else:
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
print(model)

t1 = time()
(hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK, evaluation_threads, device)
hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
print('Init: HR = %.4f, NDCG = %.4f\t [%.1f s]' % (hr, ndcg, time()-t1))

In [ ]:
# Train model
best_hr, best_ndcg, best_iter = -1, -1, -1
for epoch in range(epochs):
    t1 = time()
    # Generate training instances
    user_input, item_input, labels = get_train_instances(train, num_negatives)

    # Create DataLoader
    train_dataset = TensorDataset(torch.LongTensor(user_input), torch.LongTensor(item_input), torch.FloatTensor(labels))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Training
    model.train()
    total_loss = 0
    for batch_user, batch_item, batch_label in train_loader:
        batch_user, batch_item, batch_label = batch_user.to(device), batch_item.to(device), batch_label.to(device)
        optimizer.zero_grad()
        output = model(batch_user, batch_item).squeeze() 
        loss = criterion(output, batch_label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    t2 = time()
    
    # Evaluation
    if epoch % verbose == 0:
        model.eval()
        hits, ndcgs = evaluate_model(model, testRatings, testNegatives, topK, evaluation_threads,device)
        hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
        print(f'Iteration {epoch} [{t2-t1:.1f} s]: HR = {hr:.4f}, NDCG = {ndcg:.4f}, loss = {total_loss/len(train_loader):.4f} [{time()-t2:.1f} s]')
        if hr > best_hr:
            best_hr, best_ndcg, best_iter = hr, ndcg, epoch

print(f"End. Best Iteration {best_iter}:  HR = {best_hr:.4f}, NDCG = {best_ndcg:.4f}.")
